In [2]:
import os
import numpy as np
import pandas as pd
import soundfile as sf
import librosa
from pyroomacoustics import room
from pyroomacoustics.room import ShoeBox
from sklearn.preprocessing import StandardScaler

In [3]:
audio_original  = r'C:\Users\AL56170\Downloads\Parkinson_dataset\Parkinson_negative'

In [4]:
audio_generated = 'u'

In [5]:
snr_db=20

In [33]:
for file in os.listdir(audio_original):
    audio_path = os.path.join(audio_original,file)
    audio,sr=sf.read(audio_path)
    noise_samples=np.random.randn(len(audio))
    signal_power= np.mean(audio**2)
    noise_power = np.mean(noise_samples**2)
    scale_factor= np.sqrt(signal_power/(noise_power*(10**(snr_db/10))))
    noisy_audio = audio + scale_factor*noise_samples
    augmented_filename= f"augmented_{file}"
    augmented_filepath = os.path.join(audio_generated,augmented_filename)
    sf.write(augmented_filepath,noisy_audio,sr)

In [34]:
import librosa
time_stretch_factor=1.5
for file in os.listdir(audio_original):
    file_path = os.path.join(audio_original, file)
    audio,sr = librosa.load(file_path,sr=None)
    time_stretched= librosa.effects.time_stretch(audio, rate=time_stretch_factor)
    output_file = f"time_stretched_{file}"
    output_path = os.path.join(audio_generated, output_file)
    sf.write(output_path, time_stretched, sr)

In [38]:
pip install shutil

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil


In [42]:
import os
import shutil

def copy_files(source_folder, destination_folder):
    # Get a list of all audio files in the source folder
    audio_files = [file for file in os.listdir(source_folder) if file.endswith('.wav')]

    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    for audio_file in audio_files:
        # Get the source and destination file paths
        source_path = os.path.join(source_folder, audio_file)
        destination_path = os.path.join(destination_folder, audio_file)

        # Copy the file from the source to the destination folder
        shutil.copy(source_path, destination_path)

def main():
    # Set the paths to the source and destination folders
    source_folder = r'C:\Users\AL56170\Downloads\Parkinson_dataset\Parkinson_negative'
    destination_folder = 'u'

    # Call the copy_files function
    copy_files(source_folder, destination_folder)

if __name__ == '__main__':
    main()


In [4]:
audio_original = 'u'
target_duration = 200
for file in os.listdir(audio_original):
    if file.endswith(".wav"):  # Adjust file extensions as needed
        file_path = os.path.join(audio_original, file)
        audio,sr = librosa.load(file_path,sr=None)
        current_duration=librosa.get_duration(y=audio,sr=sr)
        if current_duration<target_duration:
            pad_length = int((target_duration - current_duration)*sr)
            padded_audio=np.pad(audio,(0,pad_length), mode='constant')
            sf.write(file_path,padded_audio,sr)
        elif current_duration>target_duration:
            num_samples = int(target_duration*sr)
            truncated_audio = audio[:num_samples]
            sf.write(file_path, truncated_audio, sr) 

In [41]:
audio_folder_positive = 'p'
audio_folder_negative = 'u'
n_mfcc = 15
hop_length=512
win_length=512
positive_features = []
negative_features = []
for file_name in os.listdir(audio_folder_positive):
    if file_name.endswith('.wav'):
        audio_path = os.path.join(audio_folder_positive, file_name)
        audio, sr = librosa.load(audio_path, sr=None)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, win_length=win_length)
        positive_features.append(mfcc.T)  # Transpose for shape compatibility

for file_name in os.listdir(audio_folder_negative):
    if file_name.endswith('.wav'):
        audio_path = os.path.join(audio_folder_negative, file_name)
        audio, sr = librosa.load(audio_path, sr=None)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, win_length=win_length)
        negative_features.append(mfcc.T)  # Transpose for shape compatibility
X = np.concatenate((positive_features, negative_features), axis=0)

print("Combined Features Shape:", X.shape)

Combined Features Shape: (290, 17227, 15)


In [42]:
X_ = X.reshape(X.shape[0],-1)

In [43]:
Y=np.concatenate((np.ones(len(positive_features)),np.zeros(len(negative_features))),axis=0)

In [138]:
Y.shape

(290,)

In [104]:
scaler = StandardScaler()
scaler.fit(X_)
X_input = scaler.transform(X_) 
joblib.dump(scaler,"scaler.pkl")

['scaler.pkl']

In [137]:
X_input = np.concatenate((X_input,X__input),axis=1)
X_input

array([[ 1.16294301, -1.59844446, -1.61548591, ...,  0.70355621,
         0.70869669,  0.72670729],
       [ 1.54753864, -1.6685437 , -1.11332476, ..., -1.83469178,
        -1.84399993, -1.86839488],
       [ 1.0678463 , -0.9446761 , -1.01356304, ...,  0.29569391,
         0.29851268,  0.30970928],
       ...,
       [-0.15965591,  1.59410214, -0.72041357, ...,  0.05792151,
         0.0593868 ,  0.06661101],
       [-1.41462278, -0.49542966,  0.44288453, ..., -0.24070817,
        -0.24094279, -0.23870767],
       [-0.60518223,  0.33194429,  0.83761257, ..., -0.15430269,
        -0.15404545, -0.15036678]])

In [109]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score,cross_val_predict

X_train,X_test, Y_train,Y_test=train_test_split(X_input,Y,test_size=0.2,random_state=40)

In [110]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report

In [111]:
classifier = SVC(kernel = 'linear',random_state = 42)
classifier.fit(X_train,Y_train)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(Y_test,y_pred)
accuracy

0.9137931034482759

In [112]:
y_pred = classifier.predict(X_test)

In [113]:
accuracy = accuracy_score(Y_test,y_pred)

In [114]:
accuracy

0.9137931034482759

In [136]:
final_dataset = np.concatenate((X_input,Y.reshape(-1,1)),axis=1)
final_dataset

array([[ 1.16294301, -1.59844446, -1.61548591, ...,  0.70869669,
         0.72670729,  1.        ],
       [ 1.54753864, -1.6685437 , -1.11332476, ..., -1.84399993,
        -1.86839488,  1.        ],
       [ 1.0678463 , -0.9446761 , -1.01356304, ...,  0.29851268,
         0.30970928,  1.        ],
       ...,
       [-0.15965591,  1.59410214, -0.72041357, ...,  0.0593868 ,
         0.06661101,  0.        ],
       [-1.41462278, -0.49542966,  0.44288453, ..., -0.24094279,
        -0.23870767,  0.        ],
       [-0.60518223,  0.33194429,  0.83761257, ..., -0.15404545,
        -0.15036678,  0.        ]])

In [116]:
np.random.shuffle(final_dataset)

In [117]:
X_final = final_dataset[:,:-1]
X_final

array([[-3.41570109e-01, -3.41586769e-01,  8.52407590e-02, ...,
        -2.56941982e+00, -2.58291032e+00, -2.61958009e+00],
       [ 9.50724483e-02, -2.15274394e-01,  6.74410760e-01, ...,
        -1.87215421e+00, -1.88167562e+00, -1.90669644e+00],
       [-1.20203018e+00, -5.65260053e-01,  4.30306882e-01, ...,
        -6.86319107e-03, -5.76667896e-03,  3.75206684e-04],
       ...,
       [-4.84483838e-02,  2.39561275e-01, -5.41850105e-02, ...,
        -5.50117033e-01, -5.52112920e-01, -5.55046970e-01],
       [-1.08461738e+00,  2.82878816e-01,  1.44368422e+00, ...,
         2.50257832e+00,  2.51795948e+00,  2.56602566e+00],
       [ 7.55871236e-01,  2.82522440e+00,  3.82806994e-02, ...,
        -7.43166171e-01, -7.46260963e-01, -7.52420212e-01]])

In [118]:
Y_final = final_dataset[:,-1]
Y_final.shape

(290,)

In [119]:
#performing cross validation
model=SVC()
model_predict=cross_val_predict(model,X_final,Y_final,cv=10)
model_score=cross_val_score(model,X_final,Y_final,cv=10)

In [120]:
model_score.mean()


0.8758620689655172

In [121]:
model_predict

array([1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1.,
       0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
       0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0.,
       0., 1., 1., 0., 1.

In [122]:
from sklearn.metrics import confusion_matrix,f1_score

confusion_result = confusion_matrix(Y_final,model_predict)
confusion_result

array([[142,   2],
       [ 34, 112]], dtype=int64)

In [38]:
audio_folder_1 = 'p'
audio_folder_2 = 'u'
spectral_centroid_features_folder1 = []
spectral_bandwidth_features_folder1 = []
spectral_contrast_features_folder1 = []

spectral_centroid_features_folder2 = []
spectral_bandwidth_features_folder2 = []
spectral_contrast_features_folder2 = []

def extract_spectral_features(audio_folder):
    spectral_centroid_features = []
    spectral_bandwidth_features = []
    spectral_contrast_features = []
    for file_name in os.listdir(audio_folder):
        if file_name.endswith('.wav'):
            audio_path = os.path.join(audio_folder, file_name)
            audio, sr = librosa.load(audio_path, sr=None)
            spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)
            spectral_centroid_features.append(spectral_centroid)
            spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)
            spectral_bandwidth_features.append(spectral_bandwidth)
            spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)
            spectral_contrast_features.append(spectral_contrast)
    spectral_centroid_features = np.array(spectral_centroid_features)
    spectral_bandwidth_features = np.array(spectral_bandwidth_features)
    spectral_contrast_features = np.array(spectral_contrast_features)
    spectral_centroid_features= np.squeeze(spectral_centroid_features)
    spectral_bandwidth_features= np.squeeze(spectral_bandwidth_features)
    spectral_contrast_features=spectral_contrast_features.reshape(spectral_contrast_features.shape[0],-1)
    return spectral_centroid_features, spectral_bandwidth_features, spectral_contrast_features
spectral_centroid_features_folder1, spectral_bandwidth_features_folder1, spectral_contrast_features_folder1 = extract_spectral_features(audio_folder_1)
spectral_centroid_features_folder2, spectral_bandwidth_features_folder2, spectral_contrast_features_folder2 = extract_spectral_features(audio_folder_2)
print("Spectral Centroid Features Folder 1 Shape:", spectral_centroid_features_folder1.shape)
print("Spectral Bandwidth Features Folder 1 Shape:", spectral_bandwidth_features_folder1.shape)
print("Spectral Contrast Features Folder 1 Shape:", spectral_contrast_features_folder1.shape)

print("Spectral Centroid Features Folder 2 Shape:", spectral_centroid_features_folder2.shape)
print("Spectral Bandwidth Features Folder 2 Shape:", spectral_bandwidth_features_folder2.shape)
print("Spectral Contrast Features Folder 2 Shape:", spectral_contrast_features_folder2.shape)




Spectral Centroid Features Folder 1 Shape: (146, 17227)
Spectral Bandwidth Features Folder 1 Shape: (146, 17227)
Spectral Contrast Features Folder 1 Shape: (146, 120589)
Spectral Centroid Features Folder 2 Shape: (144, 17227)
Spectral Bandwidth Features Folder 2 Shape: (144, 17227)
Spectral Contrast Features Folder 2 Shape: (144, 120589)


In [134]:
X_p = np.concatenate((spectral_centroid_features_folder1,spectral_bandwidth_features_folder1,spectral_contrast_features_folder1),axis=1)
X_n = np.concatenate((spectral_centroid_features_folder2,spectral_bandwidth_features_folder2,spectral_contrast_features_folder2),axis=1)
X__ = np.concatenate((X_p,X_n),axis=0)
X__.shape
X__

array([[1.08158481e+04, 1.09878844e+04, 1.09766035e+04, ...,
        1.35618636e+01, 1.35618636e+01, 1.35618636e+01],
       [1.08286775e+04, 1.04418236e+04, 1.04281058e+04, ...,
        6.38526087e+00, 6.38526087e+00, 6.38526087e+00],
       [1.04081524e+04, 1.07166999e+04, 1.07997753e+04, ...,
        1.24086801e+01, 1.24086801e+01, 1.24086801e+01],
       ...,
       [3.90713818e+03, 4.57032999e+03, 4.89727697e+03, ...,
        1.17364061e+01, 1.17364061e+01, 1.17364061e+01],
       [8.85150512e+03, 8.10808462e+03, 7.91660057e+03, ...,
        1.08920652e+01, 1.08920652e+01, 1.08920652e+01],
       [7.17603233e+03, 7.12789816e+03, 7.07805914e+03, ...,
        1.11363668e+01, 1.11363668e+01, 1.11363668e+01]])

In [135]:
scaler = StandardScaler()
scaler.fit(X__)
X__input = scaler.transform(X__) 
X__input

array([[ 1.39589149,  1.45492464,  1.34693476, ...,  0.70355621,
         0.70869669,  0.72670729],
       [ 1.40111683,  1.22697511,  1.12781945, ..., -1.83469178,
        -1.84399993, -1.86839488],
       [ 1.22983983,  1.34172045,  1.27629497, ...,  0.29569391,
         0.29851268,  0.30970928],
       ...,
       [-1.41797848, -1.22404158, -1.08165041, ...,  0.05792151,
         0.0593868 ,  0.06661101],
       [ 0.59582812,  0.25277102,  0.12451683, ..., -0.24070817,
        -0.24094279, -0.23870767],
       [-0.08658041, -0.15640143, -0.21046588, ..., -0.15430269,
        -0.15404545, -0.15036678]])

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [48]:
Y_final

array([0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0.,
       1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0.,
       1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1.,
       0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1.,
       1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.,
       1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1.,
       1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1.,
       1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0.,
       0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [64]:
X_final.shape

(290, 413448)

In [65]:
Y_final.shape


(290,)

In [ ]:
import numpy as np
from sklearn.feature_selection import mutual_info_classif

# Assuming you have 'X' with shape (n_samples, n_features) and 'y' with shape (n_samples,)
# where n_samples is the number of samples and n_features is the number of features.

# Perform Information Gain feature selection
def information_gain_feature_selection(X, y, k):
    # Calculate the Information Gain (mutual information) between each feature and the target variable
    information_gain = mutual_info_classif(X, y)

    # Sort the features based on Information Gain in descending order and select the top-k features
    selected_features_indices = np.argsort(information_gain)[::-1][:k]
    return selected_features_indices

# Set the desired number of features to select
num_features_to_select = 5000

# Perform Information Gain feature selection
selected_features_indices = information_gain_feature_selection(X_final, Y_final, num_features_to_select)



In [66]:
X_selected.shape

NameError: name 'X_selected' is not defined

In [99]:
X_train,X_test, Y_train,Y_test=train_test_split(X_final,Y_final,test_size=0.2,random_state=40)

In [132]:
classifier = SVC(kernel = 'linear',random_state = 42)
classifier.fit(X_train,Y_train)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(Y_test,y_pred)
accuracy

0.9137931034482759

In [131]:
from sklearn.metrics import classification_report
classification_result  = classification_report(Y_test,y_pred)
classification_result

'              precision    recall  f1-score   support\n\n         0.0       0.90      0.93      0.92        29\n         1.0       0.93      0.90      0.91        29\n\n    accuracy                           0.91        58\n   macro avg       0.91      0.91      0.91        58\nweighted avg       0.91      0.91      0.91        58\n'

In [124]:
confusion_result = confusion_matrix(Y_test,y_pred)
confusion_result

array([[27,  2],
       [ 3, 26]], dtype=int64)

In [125]:
#performing cross validation
classifier = SVC(kernel = 'linear',random_state = 42)
model_predict=cross_val_predict(classifier,X_final,Y_final,cv=10)
model_score=cross_val_score(model,X_final,Y_final,cv=10)

In [128]:
model_score.mean()

0.8758620689655172

In [129]:
from sklearn.metrics import classification_report
classification_result  = classification_report(Y_final,model_predict)
classification_result

'              precision    recall  f1-score   support\n\n         0.0       0.91      0.95      0.93       144\n         1.0       0.95      0.90      0.93       146\n\n    accuracy                           0.93       290\n   macro avg       0.93      0.93      0.93       290\nweighted avg       0.93      0.93      0.93       290\n'

In [133]:
joblib.dump(classifier,"parkinson_model1.pkl")

['parkinson_model1.pkl']

In [ ]:
joblib.dump(scaler1,"scaler1.pkl")

In [ ]:
joblib.dump(scaler,"scaler.pkl")

In [63]:
import joblib